In [5]:
from pathlib import Path

for path in Path('images').rglob('*.jpg'):
    print(path.name)

dog.jpg
street.jpg
farm.jpg
pool.jpg
supermarket.jpg
crossing.jpg
porsche.jpg
people_car.jpg
test-image3.jpg
me.jpg
idris.jpg
idris_2.jpg
family.jpg
face.jpg
couple.jpg
